## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo Rides</strong>

<strong>3. Pooled Rides</strong>

## Load ExMAS and MaaSSim


In [71]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
#from MaaSSim.performance import kpi_pax_pricing
#from MaaSSim.performance import kpi_veh_pricing


import logging
import matplotlib.pyplot as plt


import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Delft, Netherlands

In [72]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 100 # reuqests (and passengers)
params.nV = 10 # vehicles


## Parameters for ExMAS

In [73]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



# params.kpi = 1 (Profit Maximazation)
# params.kpi = 2 (Solo Rides)
# params.kpi = 3 (Pooled Rides)


In [74]:
params.kpi = 3


### Profit Mazimization - Begin 

In [75]:
#inData = ExMAS.main(inData, params.shareability) # create shareability graph (ExMAS) 
inData = ExMAS.main(inData, params.shareability, plot = False)

27-07-23 14:01:44-INFO-Initializing pairwise trip shareability between 100 and 100 trips.
27-07-23 14:01:44-INFO-creating combinations
27-07-23 14:01:44-INFO-9900	 nR*(nR-1)
27-07-23 14:01:45-INFO-Reduction of feasible pairs by 99.36%
27-07-23 14:01:45-INFO-Degree 2 	Completed
27-07-23 14:01:45-INFO-trips to extend at degree 2 : 202
27-07-23 14:01:45-INFO-At degree 2 feasible extensions found out of 15 searched
27-07-23 14:01:45-INFO-Degree 3 	Completed
27-07-23 14:01:45-INFO-trips to extend at degree 3 : 15
27-07-23 14:01:45-INFO-At degree 3 feasible extensions found out of 1 searched
27-07-23 14:01:45-INFO-Degree 4 	Completed
27-07-23 14:01:45-INFO-Max degree reached 4
27-07-23 14:01:45-INFO-Trips still possible to extend at degree 4 : 1


In [76]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

27-07-23 14:01:45-INFO-Matching 100 trips to 318 rides in order to min u_veh
27-07-23 14:01:45-INFO-Problem solution: Optimal. 
Total costs for single trips:         27,298 
reduced by matching to:               23,122


In [77]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],2.480500,134,1,[2.4804999999999997],"[0, 134]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1470642651, 4655063484]","[None, 0, 0]",node time req_id od 0 ...,134,1.34,2.011500,0.402300,1.6092
1,[1],3.928000,212,1,[3.928],"[4, 212]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 2623902896, 44875057]","[None, 1, 1]",node time req_id od 0 ...,212,2.12,3.186000,0.637200,2.5488
2,[2],7.446000,402,1,[7.446],"[42, 402]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1679761156, 44896436]","[None, 2, 2]",node time req_id od 0 ...,402,4.02,6.039000,1.207800,4.8312
3,[3],5.587000,302,1,[5.587],"[43, 302]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44772593, 4474353182]","[None, 3, 3]",node time req_id od 0 ...,302,3.02,4.530000,0.906000,3.6240
4,[4],3.812500,206,1,[3.8125],"[53, 206]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 315119897, 44839179]","[None, 4, 4]",node time req_id od 0 N...,206,2.06,3.091500,0.618300,2.4732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,"[51, 52, 54]",20.063775,1162,32,"[14.461225, 3.307275, 2.295275]","[1606.5, 108, 142, 105, 61, 746]","[51, 52, 54]","[52, 54, 51]",3,313,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 626009978, 44781481, 44803328, 44824147...","[None, 51, 52, 54, 52, 54, 51]",node time req_id od 0 ...,1162,11.62,12.645000,2.529000,10.1160
314,"[28, 27, 30]",20.007625,899,30,"[5.634575, 4.4936750000000005, 9.879375]","[851.5, 27, 47, 297, 74, 454]","[28, 27, 30]","[27, 28, 30]",3,314,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1570983366, 2381861250, 1570982978, 151...","[None, 28, 27, 30, 27, 28, 30]",node time req_id od 0 ...,899,8.99,12.375000,2.475000,9.9000
315,"[18, 19, 16]",17.436350,708,32,"[8.9303, 3.926925, 4.579125]","[547.0, 127, 105, 130, 180, 166]","[18, 19, 16]","[19, 16, 18]",3,315,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 2560427063, 3669138214, 1530303740, 448...","[None, 18, 19, 16, 19, 16, 18]",node time req_id od 0 ...,708,7.08,10.784250,2.156850,8.6274
316,"[18, 9, 19]",19.693850,632,31,"[8.9212, 6.777475000000001, 3.995175]","[443.0, 49, 130, 225, 76, 152]","[18, 9, 19]","[19, 9, 18]",3,316,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 2560427063, 4241243118, 3669138214, 448...","[None, 18, 9, 19, 19, 9, 18]",node time req_id od 0 ...,632,6.32,12.154500,2.430900,9.7236


In [78]:
#sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)

In [79]:
sim = simulate(params = params, inData = inData,  logger_level = logging.CRITICAL) # simulate

27-07-23 14:01:47-INFO-Matching 100 trips to 318 rides in order to min u_veh
27-07-23 14:01:48-INFO-Problem solution: Optimal. 
Total costs for single trips:         27,298 
reduced by matching to:               23,122
0.0005
27-07-23 14:01:50-CRITICAL-this is request 1 with [1] available rides.
27-07-23 14:01:50-CRITICAL-ride 1 available [1]
27-07-23 14:01:50-CRITICAL-this is request 1 with [1] still available rides.
27-07-23 14:01:50-CRITICAL-vehicle 3 has 1 choices
0.0005
27-07-23 14:01:50-CRITICAL-this is request 3 with [3] available rides.
27-07-23 14:01:50-CRITICAL-ride 3 available [1]
27-07-23 14:01:50-CRITICAL-this is request 3 with [3] still available rides.
27-07-23 14:01:50-CRITICAL-vehicle 10 has 1 choices
0.0005
27-07-23 14:01:51-CRITICAL-this is request 5 with [5] available rides.
27-07-23 14:01:51-CRITICAL-ride 5 available [1]
27-07-23 14:01:51-CRITICAL-this is request 5 with [5] still available rides.
27-07-23 14:01:51-CRITICAL-vehicle 6 has 1 choices
0.0005
27-07-23 14

In [80]:
responses = []
avg_kpi = []
idle_time = []

for i in range(3, 4):
    params.kpi = i
    #sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)
   # sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv'.format(i))
    #driver_data.loc['Cost'] = driver_data.loc['Revenue'].apply(lambda x: x*params.shareability.operating_cost)['Vehicles'] = sim.res[0].veh_exp.index
    #sim.res[0].veh_exp['ds'] = f"{i}"
    
    #responses.append(sim.res[0].veh_exp)
     
   # vehicles = sim.res[0].veh_exp.loc[sim.res[0].veh_exp["nRIDES"] > 0]
    #no_of_veh = len(vehicles)
    
    #avg_kpi.append(sim.res[0].all_kpi/no_of_veh)
    #idle_time.append(vehicles['IDLE'].sum()/no_of_veh)
    
    #_inData.sblts.rides['commission'] = _inData.sblts.rides.apply(lambda row: row.fare*sp.comm_rate, axis=1)
    

In [81]:
sim.res[0].pax_exp

,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,LOSES_PATIENCE,MEETS_DRIVER_AT_PICKUP,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,RECEIVES_OFFER,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,TRAVEL,WAIT,OPERATIONS
pax,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,615.0,0.0,0.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0
1,15.0,212.0,20.0,30.0,0.0,50.0,4.0,10.0,0,0,0,0,0,0,212.0,50.0,55.0
2,0.0,0.0,0.0,0.0,615.0,0.0,42.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0
3,15.0,302.0,20.0,30.0,0.0,142.0,43.0,10.0,0,0,0,0,0,0,302.0,142.0,55.0
4,0.0,0.0,0.0,0.0,615.0,0.0,53.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,0.0,0.0,615.0,0.0,3302.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0
96,15.0,528.0,20.0,30.0,0.0,55.0,3429.0,10.0,0,0,0,0,0,0,528.0,55.0,55.0
97,0.0,0.0,0.0,0.0,615.0,0.0,3474.0,0.0,0,0,0,0,0,0,0.0,0.0,0.0


In [82]:
sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[False]

AttributeError: 'DataFrame' object has no attribute 'LOST_PATIENCE'

In [ ]:
Fulfilled_requests = sim.res[0].pax_exp.LOSES_PATIENCE.value_counts()[False]
unFulfilled_requests = sim.res[0].pax_exp.LOSES_PATIENCE.value_counts()[True]
service_rate = Fulfilled_requests/(Fulfilled_requests+unFulfilled_requests)
print('service_rate = ', service_rate*100, '%')

In [ ]:
sim.vehs[5].rdf 

In [ ]:
sim.inData.sblts.rides

In [ ]:
pd.DataFrame(sim.vehs[4].myrides)

In [ ]:
responses = []
avg_kpi = []
idle_time = []

for i in range(1, 4):
    params.kpi = i
    #sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)
    sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv'.format(i))
    #driver_data.loc['Cost'] = driver_data.loc['Revenue'].apply(lambda x: x*params.shareability.operating_cost)['Vehicles'] = sim.res[0].veh_exp.index
    #sim.res[0].veh_exp['ds'] = f"{i}"
    
    #responses.append(sim.res[0].veh_exp)
     
   # vehicles = sim.res[0].veh_exp.loc[sim.res[0].veh_exp["nRIDES"] > 0]
    #no_of_veh = len(vehicles)
    
    #avg_kpi.append(sim.res[0].all_kpi/no_of_veh)
    #idle_time.append(vehicles['IDLE'].sum()/no_of_veh)
    
    #_inData.sblts.rides['commission'] = _inData.sblts.rides.apply(lambda row: row.fare*sp.comm_rate, axis=1)
    

In [ ]:
Fulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[False]
unFulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts()[True]
service_rate = Fulfilled_requests/(Fulfilled_requests+unFulfilled_requests)
print('service_rate = ', service_rate*100, '%')